In [ ]:
import torch
from model.modeling_UniBioseq import UniBioseqForEmbedding
from model.tokenization_UniBioseq import UBSLMTokenizer
model_file = "./load_files/BiooBang_FM"

/opt/user/hsj/anaconda3/envs/UniBioseq_torch211/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-08-01 14:51:07,778] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
#init UniBioseqForEmbedding or UniBioseqForstructureEmbedding_convbert_MRL
model_fm = UniBioseqForEmbedding.from_pretrained(model_file)
tokenizer = UBSLMTokenizer.from_pretrained(model_file)
input_seq = "MTNSQALIKTLDFQLDIQSDNESLLYDATLEARSVYNQSIRLAKEGVDWNVIPDRVAEDASLVKNTIQRVVAKALGAMENYYEYDDYNKPSHTKDGTYPLRTNYEEGYNLSLTDDGDVAFRISAKPYKHVKGVLDGDDAHLDILKTALESDAWTIGTAEALFRNGNAELHVNVTSSEGTVRDKENSRTVVGVDVNEDNVALTALSKGSVEDTVVIEFPDIKFKRHRYFTMRKRVQNSGKDSIHDTLEGREERFVRDRLHKVSRHIVEWSRQFEKPCIVFEDLKEMRDGLDYGTRMNRRLHRLPFRALQHYTSYKAAFAGIPTAWINPEYTSQCCPMCGHTERSNRHKKRFKCQSCSHQDHADRSASVNIAVKGVEEYQEWTVPALNSLPVVRKVRRQASGAVDAPTVTHDAVRGYQTDGVAGVSD"
input_ids = torch.tensor([tokenizer.encode(input_seq)])
#run model
model_fm = model_fm.to("cuda")
result_fm = model_fm(input_ids.to(model_fm.device))['hidden_states'][0][1:-1] #UniBioseqForEmbedding

No vocab_list supplied for folding model, assuming the UBL vocabulary!


In [ ]:
#print if result shape == input length
print(result_fm.shape[0]==len(input_seq))

True
